In [46]:
import dask.dataframe as ddf
import pandas as pd
import datetime
import functools
import seaborn as sns
import matplotlib.pyplot as plt

### Import 2000 standard population

In [47]:
year = 1970

In [48]:
standard_pop = pd.read_csv('2000_standard_population.csv')
standard_pop

,Age,Number,Weight
0,All ages,1000000,1.000000
1,Under 1 year,13818,0.013818
2,1-4 years,55317,0.055317
3,5-14 years,145565,0.145565
4,15-24 years,138646,0.138646
5,25-34 years,135573,0.135573
6,35-44 years,162613,0.162613
7,45-54 years,134834,0.134834
8,55-64 years,87247,0.087247
9,65-74 years,66037,0.066037


In [49]:
standard_pop = standard_pop.iloc[1: , :]
standard_pop

,Age,Number,Weight
1,Under 1 year,13818,0.013818
2,1-4 years,55317,0.055317
3,5-14 years,145565,0.145565
4,15-24 years,138646,0.138646
5,25-34 years,135573,0.135573
6,35-44 years,162613,0.162613
7,45-54 years,134834,0.134834
8,55-64 years,87247,0.087247
9,65-74 years,66037,0.066037
10,75-84 years,44842,0.044842


In [50]:
standard_pop['Age'].unique()

array(['Under 1 year', '1-4 years', '5-14 years', '15-24 years',
       '25-34 years', '35-44 years', '45-54 years', '55-64 years',
       '65-74 years', '75-84 years', '85 years and over'], dtype=object)

### Import real population data

In [51]:
pop = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/temp_bins_suicide/Population/population_detailed_1969-2020.csv', dtype = {'year': int, 'fips': str})
pop = pop.loc[:, ~pop.columns.str.contains('^Unnamed')]
pop

,year,state,fips,race,sex,age,pop
0,1969,AL,01001,white,Male,0 years,159
1,1969,AL,01001,white,Male,1-4 years,657
2,1969,AL,01001,white,Male,5-9 years,1137
3,1969,AL,01001,white,Male,10-14 years,956
4,1969,AL,01001,white,Male,15-19 years,721
...,...,...,...,...,...,...,...
15536023,2020,WY,56045,other,Female,65-69 years,21
15536024,2020,WY,56045,other,Female,70-74 years,2
15536025,2020,WY,56045,other,Female,75-79 years,2
15536026,2020,WY,56045,other,Female,80-84 years,2


In [52]:
pop['age'].unique()

array(['0 years', '1-4 years', '5-9 years', '10-14 years', '15-19 years',
       '20-24 years', '25-29 years', '30-34 years', '35-39 years',
       '40-44 years', '45-49 years', '50-54 years', '55-59 years',
       '60-64 years', '65-69 years', '70-74 years', '75-79 years',
       '80-84 years', 'over 85 years'], dtype=object)

In [53]:
age_replace = {'0 years': 'Under 1 year', '5-9 years': '5-14 years', '10-14 years': '5-14 years', 
               '15-19 years': '15-24 years', '20-24 years': '15-24 years', '25-29 years': '25-34 years',
               '30-34 years': '25-34 years', '35-39 years': '35-44 years', '40-44 years': '35-44 years', 
               '45-49 years': '45-54 years', '50-54 years': '45-54 years', '55-59 years': '55-64 years',
               '60-64 years': '55-64 years', '65-69 years': '65-74 years', '70-74 years': '65-74 years', 
               '75-79 years': '75-84 years', '80-84 years': '75-84 years', 'over 85 years': '85 years and over'}

In [54]:
pop['age'] = pop['age'].replace(age_replace)
pop.head()

,year,state,fips,race,sex,age,pop
0,1969,AL,01001,white,Male,Under 1 year,159
1,1969,AL,01001,white,Male,1-4 years,657
2,1969,AL,01001,white,Male,5-14 years,1137
3,1969,AL,01001,white,Male,5-14 years,956
4,1969,AL,01001,white,Male,15-24 years,721


In [55]:
pop_year = pop.drop(['state'], axis = 1)
pop_year = pop_year.groupby(['year', 'fips', 'race', 'sex', 'age']).sum()
pop_year = pop_year.reset_index()
pop_year.head()

,year,fips,race,sex,age,pop
0,1969,01001,black,Female,1-4 years,31
1,1969,01001,black,Female,15-24 years,238
2,1969,01001,black,Female,25-34 years,318
3,1969,01001,black,Female,35-44 years,328
4,1969,01001,black,Female,45-54 years,295


In [56]:
pop_year

,year,fips,race,sex,age,pop
0,1969,01001,black,Female,1-4 years,31
1,1969,01001,black,Female,15-24 years,238
2,1969,01001,black,Female,25-34 years,318
3,1969,01001,black,Female,35-44 years,328
4,1969,01001,black,Female,45-54 years,295
...,...,...,...,...,...,...
9230030,2020,56045,white,Male,55-64 years,556
9230031,2020,56045,white,Male,65-74 years,461
9230032,2020,56045,white,Male,75-84 years,178
9230033,2020,56045,white,Male,85 years and over,8


In [57]:
pop_year = pop_year.drop(['race', 'sex'], axis = 1)
pop_year = pop_year.groupby(['year', 'fips', 'age']).sum()
pop_year = pop_year.reset_index()
pop_year

,year,fips,age,pop
0,1969,01001,1-4 years,1588
1,1969,01001,15-24 years,2844
2,1969,01001,25-34 years,3139
3,1969,01001,35-44 years,2818
4,1969,01001,45-54 years,2046
...,...,...,...,...
1785441,2020,56045,55-64 years,857
1785442,2020,56045,65-74 years,908
1785443,2020,56045,75-84 years,405
1785444,2020,56045,85 years and over,130


### Import lung cancer data

In [58]:
cancer = pd.read_csv('Lung_Cancer_Mortality/lung_cancer_mortality/lung_cancer_mortality_' + str(year) + '.csv', dtype = {'year': int, 'county_residence': str, 'county_death': str})
cancer = cancer.loc[:, ~cancer.columns.str.contains('^Unnamed')]
cancer

,year,county_residence,state_residence,state_death,county_death,month,sex,race,age,age_range,death_cause
0,1970,36999,36,55,55049,Sep,Female,White,85 years and over,65 and older,lung cancer
1,1970,55079,55,55,55079,Aug,Male,White,55-64 years,age 25-64,lung cancer
2,1970,06037,6,6,06037,Sep,Male,White,65-74 years,65 and older,lung cancer
3,1970,54011,54,54,54011,Sep,Female,White,55-64 years,age 25-64,lung cancer
4,1970,17143,17,17,17143,Aug,Male,White,55-64 years,age 25-64,lung cancer
...,...,...,...,...,...,...,...,...,...,...,...
65115,1970,09003,9,9,09003,Nov,Female,Black,55-64 years,age 25-64,lung cancer
65116,1970,39113,39,39,39113,Dec,Male,White,55-64 years,age 25-64,lung cancer
65117,1970,41051,41,41,41051,Feb,Male,Black,45-54 years,age 25-64,lung cancer
65118,1970,18163,18,18,18163,Jan,Male,White,65-74 years,65 and older,lung cancer


In [59]:
cancer_death = cancer.drop(['county_residence', 'state_residence', 'state_death', 'age_range'], axis = 1)
cancer_death = cancer_death.groupby(['year', 'county_death', 'month', 'sex', 'race', 'age']).count()
cancer_death = cancer_death.reset_index()
cancer_death.head()

,year,county_death,month,sex,race,age,death_cause
0,1970,01001,Dec,Male,White,65-74 years,1
1,1970,01001,Jun,Male,White,65-74 years,1
2,1970,01001,Sep,Male,White,55-64 years,1
3,1970,01003,Apr,Female,Black,15-24 years,1
4,1970,01003,Apr,Male,White,55-64 years,1


In [60]:
cancer_death = cancer_death.rename(columns = {'death_cause': 'deaths'})
cancer_death

,year,county_death,month,sex,race,age,deaths
0,1970,01001,Dec,Male,White,65-74 years,1
1,1970,01001,Jun,Male,White,65-74 years,1
2,1970,01001,Sep,Male,White,55-64 years,1
3,1970,01003,Apr,Female,Black,15-24 years,1
4,1970,01003,Apr,Male,White,55-64 years,1
...,...,...,...,...,...,...,...
35534,1970,56033,Oct,Male,White,65-74 years,1
35535,1970,56037,Feb,Male,White,75-84 years,1
35536,1970,56037,Jun,Male,White,65-74 years,1
35537,1970,56037,Jun,Male,White,75-84 years,1


In [61]:
cancer_death = cancer_death.drop(['sex', 'race'], axis = 1)
cancer_death = cancer_death.groupby(['year', 'county_death', 'month', 'age']).sum()
cancer_death = cancer_death.reset_index()
cancer_death

,year,county_death,month,age,deaths
0,1970,01001,Dec,65-74 years,1
1,1970,01001,Jun,65-74 years,1
2,1970,01001,Sep,55-64 years,1
3,1970,01003,Apr,15-24 years,1
4,1970,01003,Apr,55-64 years,1
...,...,...,...,...,...
27889,1970,56033,Oct,65-74 years,1
27890,1970,56037,Feb,75-84 years,1
27891,1970,56037,Jun,65-74 years,1
27892,1970,56037,Jun,75-84 years,1


In [62]:
cancer_death['deaths'].unique()

array([ 1,  2,  3,  4,  5,  6,  9,  7, 10, 12,  8, 15, 13, 16, 11, 36, 75,
       50, 26, 14, 31, 55, 49, 43, 40, 35, 20, 53, 57, 44, 60, 65, 39, 38,
       74, 22, 34, 68, 18, 29, 58, 45, 48, 72, 33, 51, 30, 17, 21, 19, 23,
       52, 27, 46, 62, 42, 41, 28, 77, 59, 54, 47, 25, 32, 37, 86, 97, 87,
       79, 83, 71, 82, 69, 85, 66, 92, 89, 81, 63, 24])

In [63]:
cancer_death = cancer_death.drop(['month'], axis = 1)
cancer_death = cancer_death.groupby(['year', 'county_death','age']).sum()
cancer_death = cancer_death.reset_index()
cancer_death

,year,county_death,age,deaths
0,1970,01001,55-64 years,1
1,1970,01001,65-74 years,2
2,1970,01003,15-24 years,1
3,1970,01003,35-44 years,1
4,1970,01003,45-54 years,2
...,...,...,...,...
8609,1970,56033,65-74 years,3
8610,1970,56033,75-84 years,4
8611,1970,56037,65-74 years,1
8612,1970,56037,75-84 years,2


### Calculate yearly county level age adjusted rate 

In [65]:
standard_pop = standard_pop.rename(columns = {'Number  ': 'Number', '   Weight': 'Weight'})
standard_pop

,Age,Number,Weight
1,Under 1 year,13818,0.013818
2,1-4 years,55317,0.055317
3,5-14 years,145565,0.145565
4,15-24 years,138646,0.138646
5,25-34 years,135573,0.135573
6,35-44 years,162613,0.162613
7,45-54 years,134834,0.134834
8,55-64 years,87247,0.087247
9,65-74 years,66037,0.066037
10,75-84 years,44842,0.044842


In [66]:
cancer_death = cancer_death.rename(columns = {'county_death': 'fips'})
cancer_death

,year,fips,age,deaths
0,1970,01001,55-64 years,1
1,1970,01001,65-74 years,2
2,1970,01003,15-24 years,1
3,1970,01003,35-44 years,1
4,1970,01003,45-54 years,2
...,...,...,...,...
8609,1970,56033,65-74 years,3
8610,1970,56033,75-84 years,4
8611,1970,56037,65-74 years,1
8612,1970,56037,75-84 years,2


In [67]:
len(cancer_death['fips'].unique())

2621

In [68]:
fips_list = list(cancer_death['fips'].unique())
fips_list[:10]

['01001',
 '01003',
 '01005',
 '01007',
 '01009',
 '01011',
 '01013',
 '01015',
 '01017',
 '01019']

In [69]:
pop_year_current = pop_year[pop_year['year'] == year]
pop_year_current

,year,fips,age,pop
33976,1970,01001,1-4 years,1385
33977,1970,01001,15-24 years,3874
33978,1970,01001,25-34 years,3200
33979,1970,01001,35-44 years,2911
33980,1970,01001,45-54 years,1794
...,...,...,...,...
67947,1970,56045,55-64 years,533
67948,1970,56045,65-74 years,364
67949,1970,56045,75-84 years,183
67950,1970,56045,85 years and over,43


In [70]:
item = '01001'
death_fips = cancer_death[cancer_death['fips'] == item]
pop_fips = pop_year_current[pop_year_current['fips'] == item]

In [71]:
death_fips

,year,fips,age,deaths
0,1970,01001,55-64 years,1
1,1970,01001,65-74 years,2


In [72]:
pop_fips

,year,fips,age,pop
33976,1970,01001,1-4 years,1385
33977,1970,01001,15-24 years,3874
33978,1970,01001,25-34 years,3200
33979,1970,01001,35-44 years,2911
33980,1970,01001,45-54 years,1794
33981,1970,01001,5-14 years,6192
33982,1970,01001,55-64 years,1925
33983,1970,01001,65-74 years,1249
33984,1970,01001,75-84 years,547
33985,1970,01001,85 years and over,121


In [73]:
df_current = death_fips.merge(pop_fips, on = ['year', 'fips', 'age'], how = 'inner')
df_current

,year,fips,age,deaths,pop
0,1970,01001,55-64 years,1,1925
1,1970,01001,65-74 years,2,1249


In [74]:
standard_pop = standard_pop.rename(columns = {'Age': 'age'})

In [75]:
standard_pop

,age,Number,Weight
1,Under 1 year,13818,0.013818
2,1-4 years,55317,0.055317
3,5-14 years,145565,0.145565
4,15-24 years,138646,0.138646
5,25-34 years,135573,0.135573
6,35-44 years,162613,0.162613
7,45-54 years,134834,0.134834
8,55-64 years,87247,0.087247
9,65-74 years,66037,0.066037
10,75-84 years,44842,0.044842


In [ ]:
df_current = df_current.merge(standard_pop, on = {'fips'})

In [ ]:
def age_adjusted_rate(df, df_pop):
    for fips in fips_list:
        df_fips = df[df['fips'] == fips]
        pop_fips = df_pop[df_pop]
        
        
    dict1 = {}
    df = df.groupby(['year','age_range']).sum()
    df = df.reset_index()
    df['cube_rate'] = (df['deaths'] / df['pop']) * 100000
    df = df.merge(standard_pop, on = ['age_range'], how = 'inner')
    df['cube_rate_weight'] = df['cube_rate'] * df['Weight']
    age_adjusted_rate = round(sum(df['cube_rate_weight']),1)
    key = df_type + '_Rate'
    dict1[key] = age_adjusted_rate
    print('age_adjusted_rate for ' + df_type + ' :' + str(age_adjusted_rate))
    return dict1

In [19]:
death_white = cancer_death[cancer_death['race'] == 'White']
death_black = cancer_death[cancer_death['race'] == 'Black']

In [20]:
sum(cancer_death['deaths'])

65108

In [21]:
sum(death_white['deaths'])

58540

In [22]:
sum(death_black['deaths'])

6232

In [23]:
all_male = cancer_death[cancer_death['sex'] == 'Male']
all_female = cancer_death[cancer_death['sex'] == 'Female']

In [24]:
sum(all_male['deaths'])

52753

In [25]:
sum(all_female['deaths'])

12355

In [26]:
white_male = death_white[death_white['sex'] == 'Male']
white_female = death_white[death_white['sex'] == 'Female']

In [27]:
sum(white_male['deaths'])

47352

In [28]:
sum(white_female['deaths'])

11188

In [29]:
black_male = death_black[death_black['sex'] == 'Male']
black_female = death_black[death_black['sex'] == 'Female']

In [30]:
sum(black_male['deaths'])

5123

In [31]:
sum(black_female['deaths'])

1109

In [32]:
def printDeathCount2(df):
    data = {}
    all_deaths = sum(df['deaths'])
    print('All Races Total: ' + str(all_deaths))
    data['All_Races_Total_Count'] = all_deaths
    
    all_male = df[df['sex'] == 'Male']
    all_female = df[df['sex'] == 'Female']
    all_male_death = sum(all_male['deaths'])
    all_female_death = sum(all_female['deaths'])
    print('All Races Male: ' + str(all_male_death))
    print('All Races Female: ' + str(all_female_death))
    data['All_Races_Male_Count'] = all_male_death
    data['All_Races_Female_Count'] = all_female_death
    
    death_white = df[df['race'] == 'White']
    death_black = df[df['race'] == 'Black']
    white_male = death_white[death_white['sex'] == 'Male']
    white_female = death_white[death_white['sex'] == 'Female']
    white_deaths = sum(death_white['deaths'])
    white_male_death = sum(white_male['deaths'])
    white_female_death = sum(white_female['deaths'])
    print('White Total: ' + str(white_deaths))
    print('White Male: ' + str(white_male_death))
    print('White Female: ' + str(white_female_death))
    data['White_Total_Count'] = white_deaths
    data['White_Male_Count'] = white_male_death
    data['White_Female_Count'] = white_female_death
    
    
    black_male = death_black[death_black['sex'] == 'Male']
    black_female = death_black[death_black['sex'] == 'Female']
    black_deaths = sum(death_black['deaths'])
    black_male_death = sum(black_male['deaths'])
    black_female_death = sum(black_female['deaths'])
    print('Black Total: ' + str(black_deaths))
    print('Black Male: ' + str(black_male_death))
    print('Black Female: ' + str(black_female_death))
    data['Black_Total_Count'] = black_deaths
    data['Black_Male_Count'] = black_male_death
    data['Black_Female_Count'] = black_female_death
    
    return data

In [33]:
data = printDeathCount2(cancer_death)

All Races Total: 65108
All Races Male: 52753
All Races Female: 12355
White Total: 58540
White Male: 47352
White Female: 11188
Black Total: 6232
Black Male: 5123
Black Female: 1109


In [34]:
data

{'All_Races_Total_Count': 65108,
 'All_Races_Male_Count': 52753,
 'All_Races_Female_Count': 12355,
 'White_Total_Count': 58540,
 'White_Male_Count': 47352,
 'White_Female_Count': 11188,
 'Black_Total_Count': 6232,
 'Black_Male_Count': 5123,
 'Black_Female_Count': 1109}

### Calculate age adjusted rates

In [35]:
pop_year['race'].unique()

array(['black', 'other', 'white'], dtype=object)

In [36]:
race_replace = {'white': 'White', 'black': 'Black', 'other': 'Other'}
pop_year = pop_year.replace(race_replace)
pop_year

,year,race,sex,age_range,pop
0,1969,Black,Female,65 and older,769920
1,1969,Black,Female,age 0-24,5679845
2,1969,Black,Female,age 25-64,4162512
3,1969,Black,Male,65 and older,593563
4,1969,Black,Male,age 0-24,5409315
...,...,...,...,...,...
931,2020,White,Female,age 0-24,33811534
932,2020,White,Female,age 25-64,59723469
933,2020,White,Male,65 and older,19428990
934,2020,White,Male,age 0-24,35476833


In [37]:
cancer_death = cancer_death.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
cancer_death

,year,sex,race,age_range,deaths,pop
0,2019,Female,Black,65 and older,4553,2901018
1,2019,Female,Black,age 0-24,5,7735808
2,2019,Female,Black,age 25-64,2011,11452879
3,2019,Female,Other,65 and older,1638,1537895
4,2019,Female,Other,age 0-24,1,4096726
5,2019,Female,Other,age 25-64,580,7057140
6,2019,Female,White,65 and older,43254,22887775
7,2019,Female,White,age 0-24,9,34033757
8,2019,Female,White,age 25-64,12759,59372188
9,2019,Male,Black,65 and older,5657,1964945


In [38]:
standard_pop

,age_range,Number,Weight
0,65 and older,126387,0.126387
1,age 0-24,353346,0.353346
2,age 25-64,520267,0.520267


In [39]:
standard_pop = standard_pop.rename(columns = {'Number  ': 'Number', '   Weight': 'Weight'})

In [40]:
cancer_all_races = cancer_death.drop(['sex', 'race'], axis = 1)
cancer_all_races = cancer_all_races.groupby(['year','age_range']).sum()
cancer_all_races = cancer_all_races.reset_index()
cancer_all_races

,year,age_range,deaths,pop
0,2019,65 and older,105771,49283871
1,2019,age 0-24,30,89271463
2,2019,age 25-64,34015,155096538


In [41]:
cancer_all_races['cube_rate'] = (cancer_all_races['deaths'] / cancer_all_races['pop']) * 100000
cancer_all_races = cancer_all_races.merge(standard_pop, on = ['age_range'], how = 'inner')
cancer_all_races

,year,age_range,deaths,pop,cube_rate,Number,Weight
0,2019,65 and older,105771,49283871,214.615853,126387,0.126387
1,2019,age 0-24,30,89271463,0.033605,353346,0.353346
2,2019,age 25-64,34015,155096538,21.931502,520267,0.520267


In [42]:
cancer_all_races.columns

Index(['year', 'age_range', 'deaths', 'pop', 'cube_rate', 'Number', 'Weight'], dtype='object')

In [43]:
cancer_all_races['cube_rate_weight'] = cancer_all_races['cube_rate'] * cancer_all_races['Weight']
cancer_all_races

,year,age_range,deaths,pop,cube_rate,Number,Weight,cube_rate_weight
0,2019,65 and older,105771,49283871,214.615853,126387,0.126387,27.124654
1,2019,age 0-24,30,89271463,0.033605,353346,0.353346,0.011874
2,2019,age 25-64,34015,155096538,21.931502,520267,0.520267,11.410237


In [44]:
dict_all_race = {}
dict_all_race['All_Races_Total_Rate'] = round(sum(cancer_all_races['cube_rate_weight']),1)
dict_all_race

{'All_Races_Total_Rate': 38.5}

In [45]:
data.update(dict_all_race)

In [46]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5}

In [47]:
def age_adjusted_rate(df, df_type):
    dict1 = {}
    df = df.groupby(['year','age_range']).sum()
    df = df.reset_index()
    df['cube_rate'] = (df['deaths'] / df['pop']) * 100000
    df = df.merge(standard_pop, on = ['age_range'], how = 'inner')
    df['cube_rate_weight'] = df['cube_rate'] * df['Weight']
    age_adjusted_rate = round(sum(df['cube_rate_weight']),1)
    key = df_type + '_Rate'
    dict1[key] = age_adjusted_rate
    print('age_adjusted_rate for ' + df_type + ' :' + str(age_adjusted_rate))
    return dict1

In [48]:
all_male

,year,sex,race,age_range,deaths
9,2019,Male,Black,65 and older,5657
10,2019,Male,Black,age 0-24,5
11,2019,Male,Black,age 25-64,2889
12,2019,Male,Other,65 and older,1918
13,2019,Male,Other,age 25-64,696
14,2019,Male,White,65 and older,48751
15,2019,Male,White,age 0-24,10
16,2019,Male,White,age 25-64,15080


In [49]:
all_female

,year,sex,race,age_range,deaths
0,2019,Female,Black,65 and older,4553
1,2019,Female,Black,age 0-24,5
2,2019,Female,Black,age 25-64,2011
3,2019,Female,Other,65 and older,1638
4,2019,Female,Other,age 0-24,1
5,2019,Female,Other,age 25-64,580
6,2019,Female,White,65 and older,43254
7,2019,Female,White,age 0-24,9
8,2019,Female,White,age 25-64,12759


In [50]:
all_male = all_male.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
all_male = all_male.drop(['sex', 'race'], axis = 1)
all_male_rate = age_adjusted_rate(all_male, 'All_Races_Male')

age_adjusted_rate for All_Races_Male :45.0


In [51]:
all_male_rate

{'All_Races_Male_Rate': 45.0}

In [52]:
data.update(all_male_rate)

In [53]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0}

In [54]:
all_female = all_female.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
all_female = all_female.drop(['sex', 'race'], axis = 1)
all_female_rate = age_adjusted_rate(all_female, 'All_Races_Female')

age_adjusted_rate for All_Races_Female :33.1


In [55]:
all_female_rate

{'All_Races_Female_Rate': 33.1}

In [56]:
data.update(all_female_rate)

In [57]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1}

In [58]:
death_white

,year,sex,race,age_range,deaths
6,2019,Female,White,65 and older,43254
7,2019,Female,White,age 0-24,9
8,2019,Female,White,age 25-64,12759
14,2019,Male,White,65 and older,48751
15,2019,Male,White,age 0-24,10
16,2019,Male,White,age 25-64,15080


In [59]:
death_white = death_white.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
death_white = death_white.drop(['sex', 'race'], axis = 1)
death_white_rate = age_adjusted_rate(death_white, 'White_Total')

age_adjusted_rate for White_Total :40.0


In [60]:
death_white_rate

{'White_Total_Rate': 40.0}

In [61]:
data.update(death_white_rate)

In [62]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1,
 'White_Total_Rate': 40.0}

In [63]:
death_black = death_black.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
death_black = death_black.drop(['sex', 'race'], axis = 1)
death_black_rate = age_adjusted_rate(death_black, 'Black_Total')

age_adjusted_rate for Black_Total :38.2


In [64]:
death_black_rate

{'Black_Total_Rate': 38.2}

In [65]:
data.update(death_black_rate)

In [66]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1,
 'White_Total_Rate': 40.0,
 'Black_Total_Rate': 38.2}

In [67]:
white_male

,year,sex,race,age_range,deaths
14,2019,Male,White,65 and older,48751
15,2019,Male,White,age 0-24,10
16,2019,Male,White,age 25-64,15080


In [68]:
white_male = white_male.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
white_male = white_male.drop(['sex', 'race'], axis = 1)
white_male_rate = age_adjusted_rate(white_male, 'White_Male')

age_adjusted_rate for White_Male :45.8


In [69]:
white_male_rate

{'White_Male_Rate': 45.8}

In [70]:
data.update(white_male_rate)

In [71]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1,
 'White_Total_Rate': 40.0,
 'Black_Total_Rate': 38.2,
 'White_Male_Rate': 45.8}

In [72]:
white_female = white_female.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
white_female = white_female.drop(['sex', 'race'], axis = 1)
white_female_rate = age_adjusted_rate(white_female, 'White_Female')

age_adjusted_rate for White_Female :35.1


In [73]:
white_female_rate

{'White_Female_Rate': 35.1}

In [74]:
data.update(white_female_rate)

In [75]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1,
 'White_Total_Rate': 40.0,
 'Black_Total_Rate': 38.2,
 'White_Male_Rate': 45.8,
 'White_Female_Rate': 35.1}

In [76]:
black_male = black_male.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
black_male = black_male.drop(['sex', 'race'], axis = 1)
black_male_rate = age_adjusted_rate(black_male, 'Black_Male')

age_adjusted_rate for Black_Male :50.9


In [77]:
black_male_rate

{'Black_Male_Rate': 50.9}

In [78]:
data.update(black_male_rate)

In [79]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1,
 'White_Total_Rate': 40.0,
 'Black_Total_Rate': 38.2,
 'White_Male_Rate': 45.8,
 'White_Female_Rate': 35.1,
 'Black_Male_Rate': 50.9}

In [80]:
black_female = black_female.merge(pop_year, on = ['year', 'race', 'sex', 'age_range'], how = 'inner')
black_female = black_female.drop(['sex', 'race'], axis = 1)
black_female_rate = age_adjusted_rate(black_female, 'Black_Female')

age_adjusted_rate for Black_Female :29.0


In [81]:
black_female_rate

{'Black_Female_Rate': 29.0}

In [82]:
data.update(black_female_rate)

In [83]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1,
 'White_Total_Rate': 40.0,
 'Black_Total_Rate': 38.2,
 'White_Male_Rate': 45.8,
 'White_Female_Rate': 35.1,
 'Black_Male_Rate': 50.9,
 'Black_Female_Rate': 29.0}

In [84]:
data['year'] = year

In [85]:
data

{'All_Races_Total_Count': 139816,
 'All_Races_Male_Count': 75006,
 'All_Races_Female_Count': 64810,
 'White_Total_Count': 119863,
 'White_Male_Count': 63841,
 'White_Female_Count': 56022,
 'Black_Total_Count': 15120,
 'Black_Male_Count': 8551,
 'Black_Female_Count': 6569,
 'All_Races_Total_Rate': 38.5,
 'All_Races_Male_Rate': 45.0,
 'All_Races_Female_Rate': 33.1,
 'White_Total_Rate': 40.0,
 'Black_Total_Rate': 38.2,
 'White_Male_Rate': 45.8,
 'White_Female_Rate': 35.1,
 'Black_Male_Rate': 50.9,
 'Black_Female_Rate': 29.0,
 'year': 2019}

In [86]:
df = pd.DataFrame([data])
df

,All_Races_Total_Count,All_Races_Male_Count,All_Races_Female_Count,White_Total_Count,White_Male_Count,White_Female_Count,Black_Total_Count,Black_Male_Count,Black_Female_Count,All_Races_Total_Rate,All_Races_Male_Rate,All_Races_Female_Rate,White_Total_Rate,Black_Total_Rate,White_Male_Rate,White_Female_Rate,Black_Male_Rate,Black_Female_Rate,year
0,139816,75006,64810,119863,63841,56022,15120,8551,6569,38.5,45.0,33.1,40.0,38.2,45.8,35.1,50.9,29.0,2019


In [87]:
df.columns

Index(['All_Races_Total_Count', 'All_Races_Male_Count',
       'All_Races_Female_Count', 'White_Total_Count', 'White_Male_Count',
       'White_Female_Count', 'Black_Total_Count', 'Black_Male_Count',
       'Black_Female_Count', 'All_Races_Total_Rate', 'All_Races_Male_Rate',
       'All_Races_Female_Rate', 'White_Total_Rate', 'Black_Total_Rate',
       'White_Male_Rate', 'White_Female_Rate', 'Black_Male_Rate',
       'Black_Female_Rate', 'year'],
      dtype='object')

In [88]:
column_valid = ['year', 'All_Races_Total_Count', 'All_Races_Total_Rate', 'All_Races_Male_Count', 'All_Races_Male_Rate', 'All_Races_Female_Count', 'All_Races_Female_Rate', 'White_Total_Count', 'White_Total_Rate',
               'White_Male_Count', 'White_Male_Rate', 'White_Female_Count', 'White_Female_Rate', 'Black_Total_Count', 'Black_Total_Rate', 'Black_Male_Count', 'Black_Male_Rate', 'Black_Female_Count', 'Black_Female_Rate']
df_valid = df[column_valid]
df_valid

,year,All_Races_Total_Count,All_Races_Total_Rate,All_Races_Male_Count,All_Races_Male_Rate,All_Races_Female_Count,All_Races_Female_Rate,White_Total_Count,White_Total_Rate,White_Male_Count,White_Male_Rate,White_Female_Count,White_Female_Rate,Black_Total_Count,Black_Total_Rate,Black_Male_Count,Black_Male_Rate,Black_Female_Count,Black_Female_Rate
0,2019,139816,38.5,75006,45.0,64810,33.1,119863,40.0,63841,45.8,56022,35.1,15120,38.2,8551,50.9,6569,29.0


In [89]:
df_valid.to_csv('age_adjusted/age_adjusted_' + str(year) + '.csv')